In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import scipy.io as scio

In [3]:
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.utils.tensorboard import SummaryWriter

In [4]:
import os
import sys
import datetime
import glob

In [35]:
# Add module path to sys
module_path = os.path.abspath(os.path.join('../..'))
print("System path: "+module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
import micro_dl.torch_unet.networks.Unet2D as Unet2D
import micro_dl.torch_unet.networks.Unet25D as Unet25D
import micro_dl.torch_unet.networks.layers.ConvBlock2D as ConvBlock2D
import micro_dl.torch_unet.networks.layers.ConvBlock3D as ConvBlock3D
import micro_dl.torch_unet.utils.dataset as ds
import micro_dl.torch_unet.utils.io as io

System path: /home/christian.foley/virtual_staining/microDL


### Data Acquisition

In [36]:
data_directory = '/hpc/projects/compmicro/projects/virtualstaining/MBL_DL_image_translation/data/'

In [37]:
all_data = {}
train_samples = {}
test_samples = {}

In [38]:
for key in io.unique_tags(data_directory):
    measurements = glob.glob(data_directory + '*' + key)
    measurements.sort()
    num = np.random.uniform(0,1)
    if num < 0.9:
        train_samples[key] = measurements
    else:
        test_samples[key] = measurements

In [39]:
print(len(train_samples), len(test_samples))

158 22


### TensorBoard Unet2D

In [40]:
#define dataloaders
batch_size = 8

train_dataset = ds.MyDataset(train_samples, transform = transforms.Compose([ds.Resize(), ds.Normalize(), ds.RandFlip(), ds.toTensor()]))
train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=False)

In [41]:
#define model
model = Unet2D.Unet2d(in_channels = 1, out_channels = 1, residual = True, task = 'recon')
#_ = model.model().cuda()
_ = model.cuda()

In [42]:
#test model functionality
out = model(next(iter(train_dataloader))['image'])
print(out.shape)

torch.Size([8, 1, 256, 256])


/home/christian.foley/.conda/envs/pt_staining/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


In [14]:
#make graph
now = str(datetime.datetime.now()).replace(' ', '_').replace(':','_').replace('-','_')[:-10]
save_folder = f'/hpc/projects/compmicro/projects/virtualstaining/torch_microDL_models/Unet2d/tensorboard_{model.__name__()}_{now}'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

writer = SummaryWriter(log_dir = save_folder)
nxt = next(iter(train_dataloader))
images, labels = nxt['image'], nxt['target']

grid = torchvision.utils.make_grid(images)
writer.add_image('images', grid, 0)
writer.add_graph(model, images)
print(f'saved at {save_folder}')
writer.close()

saved at /hpc/projects/compmicro/projects/virtualstaining/torch_microDL_models/Unet2d/tensorboard_Unet2d_2022_08_30_17_24


### TensorBoard Unet2.5D

In [60]:
#define dataloaders
batch_size = 5

train_dataset = ds.MyDataset(train_samples, transform = transforms.Compose([ds.Resize(), ds.Normalize(), ds.RandFlip(), ds.toTensor()]))
train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=False)

In [61]:
#define model
model_2 = Unet25D.Unet25d(in_channels = 1, out_channels = 1, in_stack_depth = 5, out_stack_depth = 1, residual = True, task = 'reg')

#_ = model.model().cuda()
_ = model_2.cuda()

In [69]:
input_ = next(iter(train_dataloader))['image']
images = torch.transpose(input_, 0, 1).unsqueeze(0)
images.shape

torch.Size([1, 1, 5, 256, 256])

In [70]:
#test model functionality
out = model_2(images)
print(out.shape)

torch.Size([1, 1, 1, 256, 256])


In [73]:
#make graph
now = str(datetime.datetime.now()).replace(' ', '_').replace(':','_').replace('-','_')[:-10]
save_folder = f'/hpc/projects/compmicro/projects/virtualstaining/torch_microDL_models/Unet25d/tensorboard_{model_2.__name__()}_{now}'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

writer = SummaryWriter(log_dir = save_folder)

# grid = torchvision.utils.make_grid(images)
# writer.add_image('images', grid, 0)
writer.add_graph(model_2, images)
print(f'saved at {save_folder}')
writer.close()

saved at /hpc/projects/compmicro/projects/virtualstaining/torch_microDL_models/Unet25d/tensorboard_Unet25d_2022_08_30_19_39
